# MA3J8 Approximation Theory and Applications 

## 03b - Algebraic Polynomials, Applications




In [ ]:
using SoftGlobalScope, LinearAlgebra, LaTeXStrings, Plots
gr();

In [ ]:
"Chebyshev Nodes for degree N interpolant"
chebnodes(N) = [ cos(j*π/N) for j = N:-1:0 ]

"""
Barycentric interpolation with a Chebyshev grid with N grid points.
The interpolant is evaluated at points `x`.
"""
function bary(f::Function, N, x)
    X = chebnodes(N)
    F = f.(X)
    return bary(F, x; X=X)
end

function bary(F::Vector, x; X = chebnodes(length(F)-1))
    N = length(F)-1
    p = 0.5 * ( F[1] ./ (x .- X[1]) + (-1)^N * F[N+1] ./(x .- X[N+1]) )
    q = 0.5 * (1.0 ./ (x .- X[1]) + (-1)^N ./ (x .- X[N+1]))
    for n = 1:N-1
        p += (-1)^n * F[n+1] ./ (x .- X[n+1])
        q += (-1)^n ./ (x .- X[n+1])
    end 
    return p ./ q    
end

"""
generate a grid on which to plot errors; this is chosen to avoid 
any grid points since barycentric interpolation is not defined 
on those.
"""
errgrid(Np) = range(-1+0.0123, stop=1-0.00321, length=Np)

## 03-4 Applications

### Evaluating a "special function"

Special functions are functions such as $\exp(x), \sin(x), \cos(x), \dots$, the Bessel functions, $\Gamma$ function, Airy, and many more. Efficient and stable numerical evaluation of such functions is a mostly solved and well-understood problem. Nevertheless it is useful to see what kind of ideas might be involved. Here, we will just use polynomial interpolation of a Taylor series, but of course in practise one uses much more sophisticated techniques (more on that later).

For simplicity, let us just consider the `sin` function. We can obtain a decent approxiation using Taylor series. Then we interpolate the Taylor series to get a Chebvyshev interpolant. Note that in principle we only need to evaluate `sin` in $[-\pi/2, \pi/2]$ as all other cases can be reduced to shifting and reflection.

In [ ]:
tsin(N, x) = imag(sum((im*x)^n / factorial(n) for n = 0:N))

xx = range(-pi/2, stop=pi/2, length=1000)
println("Taylor Expansion:")
for N in [7, 11, 15, 19]
    errN = maximum(abs(sin(x) - tsin(N,x)) for x in xx) 
    println(" N = $N => err = $errN")
end 

This suggests that `tsin(20, x)` is a machine-precision approximation to `sin`. Now we can check how many points we need with a Chebyshev interpolant.

In [ ]:
scal_tsin = s -> tsin(20, s*π/2)
println("Chebyshev Interpolant:")
for N in [7, 9, 11, 15]
    errN = norm(bary(scal_tsin, N, xx*2/π) - sin.(xx), Inf)
    println(" N = $N => err = $errN")
end

In [ ]:
# we can do even better by approximating only on the 
# interval [0, pi/2]
xx = range(0+3.21e-12, stop=pi/2, length=333)
scal_tsin1 = s -> sin((1 + s)*π/4)
println("Chebyshev Interpolant:")
for N in [5, 8, 11, 13]
    errN = norm(bary(scal_tsin1, N, xx*4/π.-1) - sin.(xx), Inf)
    println(" N = $N => err = $errN")
end

This may seem like a small improvement, but a factor 2/3 in the evaluation cost would in fact represent a phenomenal gain in computing speed. The explanatio of this gain can be easily visualised: the taylor polynomial optimises the error in the origin while the Chebyshev interpolant distributes it more uniformly (but not uniformly enough, which is why it is not optimal; see later).

In [ ]:
N = 13
xp = range(-pi/2, stop=pi/2, length=400)
terr = tsin.(N, xp) - sin.(xp)
cerr = bary(scal_tsin, N, xp*2/π) - sin.(xp)
plot(xp, [terr, cerr], lw=2, label = ["Taylor-error", "Chebyshev-error"],
         ylim = [-2e-13, 2e-13])

In [ ]:
using BenchmarkTools

function bary1(F::Vector{T}, X::Vector{T}, x::T) where {T}
    N = length(F)-1
    σ = 1
    p = zero(T)
    q = zero(T)
    for n = 1:N-1
        σ *= -1 
        p += σ * F[n+1] / (x - X[n+1])
        q += σ / (x - X[n+1])
    end 
    σ *= -1 
    p += ( F[1] / (x - X[1]) + σ * F[N+1] / (x - X[N+1]) ) / 2
    q += ( 1 / (x - X[1]) + σ / (x - X[N+1])  ) / 2
    return p ./ q    
end

function runn(f, x, N)
    y = f(x)
    for n = 2:N
        x += 1e-12 
        y += f(x)
    end
    return y
end

N = 13 
scal_sin = s -> sin((1 + s)*π/4)
const Xsin = chebnodes(N)
const Fsin = scal_sin.(Xsin)

function cheb_sin(x)
    s = 4/π * x - 1
    return bary1(Fsin, Xsin, s)
end 

x0 = rand() * pi/2
@btime runn($sin, $x0, 1_000_000)
@btime runn($cheb_sin, $x0, 1_000_000)

In [ ]:
using FFTW
# we first implement the fast chebyshev transform 

revchebnodes(N) = [ cos(j*π/N) for j = 0:N ]

function fct(F)
    N = length(F)-1
    G = [F; F[N:-1:2]]
    Ĝ = real.(ifft(G))
    return [Ĝ[1]; 2 * Ĝ[2:N]; Ĝ[N+1]]
end 


function eval_chebpoly(F̃, x)
    N = length(F̃)-1
    Told = one(x)
    #     if N == 0; return Told * F̃[1]; end 
    Tnew = x 
    p = Told * F̃[1] + Tnew * F̃[2]
    #     if N == 1; return p; end 
    for n = 2:N 
        Toldold = Told 
        Told = Tnew 
        Tnew = 2 * x * Told - Toldold
        p += F̃[n+1] * Tnew
    end 
    return p 
end


In [ ]:
const F̃sin = fct(Fsin)
function cheb_sin2(x)
    s = 4/π * x - 1
    return eval_chebpoly(F̃sin, s)
end 

x0 = rand() * pi/2
@btime runn($sin, $x0, 1_000_000)
@btime runn($cheb_sin, $x0, 1_000_000)
@btime runn($cheb_sin2, $x0, 1_000_000)

### Evaluating a Matrix Function 

Consider a discrete Laplacian-like matrix, 
$$
    H = \frac{1}{2}\begin{pmatrix}
        0 & 1      &        &        & \\ 
        1 & 0      & 1      &        &  \\ 
          & \ddots & \ddots & \ddots &  \\ 
          &        &      1 &  0     & 1 \\ 
          &        &        &      1 & 0
    \end{pmatrix}
$$
One can readily see that $\sigma(H) \subset [-1,1]$.

In [ ]:
using SparseArrays
Hfun(d) = spdiagm( -1 => ones(d-1)/2, 1 => ones(d-1)/2 )

println("H(5) = ")
display(Matrix(Hfun(5)))

print("σ(H(100)) ⊂ ")
println(extrema(eigvals(Matrix(Hfun(100)))))
;

We wish to evaluate $f_\beta(H)$ where $f_\beta$ is the Fermi-Dirac function 
$$
    f_\beta(z) = \Big( 1 + e^{\beta z} \Big)^{-1}
$$
We construct a Chebyshev interpolant, then use the Chebyshev transform to obtain the Chebyshev coefficients, which will then allow us to evaluate $f_\beta(H)$ via the Chebyshev basis recursion as a series of Matrix multiplications.

In [ ]:
f_fermi(β, x) = 1/(1 + exp(β*x))

# a little test
β = 10+rand()
xx = range(-1, stop=1, length=1000)
for N in [11, 21, 31, 41]
    F = f_fermi.(β, revchebnodes(N))
    F̃ = fct(F)
    err = norm(f_fermi.(β, xx) - [eval_chebpoly(F̃, x) for x in xx])
    println("N = $N => err = $err")
end

In [ ]:
A = Matrix(Hfun(10))
A = exp(A)

In [ ]:
# and now we can use this to evaluat a matrix function 
# ------------------------------------------------------

# exact matrix function
f_fermi_mat(β, H) = pinv(I + exp(β * Matrix(H)))

# using the chebyshev expansion 
function f_fermi_mat_cheb(β, H, N)
    F = f_fermi.(β, revchebnodes(N))
    F̃ = fct(F)
    eval_chebpoly(F̃, H)
end

In [ ]:
β, d = 10.0, 1000
# ---------------
A = Hfun(d)
fH = f_fermi_mat(β, A)
for N in [11, 21, 31, 41]
    fH_N = f_fermi_mat_cheb(β, A, N)
    err = norm(fH_N - fH, Inf)
    println("N = $N => err = $err")
end

In [ ]:
println("Runtime f_fermi_mat")
for n = 1:3 
    @time f_fermi_mat(β, A)
end
println("Runtime f_fermi_mat_cheb, N = 11")
for n = 1:3 
    @time f_fermi_mat_cheb(β, A, 11)
end
println("Runtime f_fermi_mat_cheb, N = 31")
for n = 1:3 
    @time f_fermi_mat_cheb(β, A, 31)
end
println("""There probably further optimisations possible in `eval_chebpoly`;
           in particular some allocations can be avoided.""")

### Solving a BVP 

As a final example, we look at how to solve boundary value problems using Chebyshev polynomials. There is an excellent Julia package, `ApproxFun.jl` that builds on the kind of ideas we discussed - and takes them much much further. So instead of putting together our own little toy code we will show how to use `ApproxFun.jl`.

Consider the BVP 
$$
    \epsilon u'' + 6 (1-x^2). u' + u^2 = 1,  
$$
with boundary conditions $u(-1) = 1, u(1) = -1/2$.

In [ ]:
using ApproxFun
x = Fun()  # defines the identity function x -> x
N = u -> [u(-1)-1, 
          u(1)+0.5, 
          0.01 * u'' + 6 * (1-x^2) * u' + u^2 - 1]
u = newton(N, 0*x)
@show typeof(u)
@show u.space
@show length(u.coefficients)
@show norm(N(u))
plot(u; lw = 2, label = "Solution to BVP")